# MAP 512 : Tatouage numérique par code Tardos

In [1]:
import numpy as np
import numpy.random as npr

m = 15
n = 30
t = 0.1

## Code de Tardos
Cette section propose plusieurs fonctions pour mettre en application la méthode proposée par Tardos.
- *matrice_identifiants* calcule une matrice de $n$ identifiants de taille $m$ par la méthode de Tardos. Les paramètres $p_i$ de Bernouilli de chaque bit de l'identifiant sont tirés selon la loi de densité $f(p) = \frac{Cte}{\sqrt{p(1-p)}}\mathbb{1}_{[t,1-t]}(p)$. On doit donc avoir $t<\frac{1}{2}$.
- *score_tardos* calcule le score de Tardos de l'identifiant $x$ par rapport à $y$ sachant la distribution des $p_i$.

### Calcul de la fonction de répartition de la loi de densité $f$
Le chagement de variable $q=\sqrt{p}$ donne :
$$\int_t^x{f\left(p\right)dp} = \int_t^x{\frac{Cte\cdot dp}{\sqrt{p(1-p)}}} = \int_\sqrt{t}^\sqrt{x}{\frac{Cte\cdot dq}{2\sqrt{(1-q^2)}}} = 2\cdot Cte\left[Arcsin\left(\sqrt{x}\right)-Arcsin\left(\sqrt{t}\right)\right]$$
Donc $Cte = \frac{1}{2\left[Arcsin\left(\sqrt{1-t}\right)-Arcsin\left(\sqrt{t}\right)\right]}$
et la fonction de répartition de cette loi vaut $F(x)=\frac{Arcsin\left(\sqrt{x}\right)-Arcsin\left(\sqrt{t}\right)}{Arcsin\left(\sqrt{1-t}\right)-Arcsin\left(\sqrt{t}\right)}$ sur $[t,1-t]$.  
Puis $F^{-1}(u)=sin^2\left[Arcsin\left(\sqrt{t}\right)+u\cdot\left(Arcsin\left(\sqrt{1-t}\right)-Arcsin\left(\sqrt{t}\right)\right)\right]$.

In [33]:
def matrice_identifiants(n,m,t):
    # On doit avoir t < 0.5
    u = npr.rand(m)
    p = np.sin(np.arcsin(np.sqrt(t)) + u*(np.arcsin(np.sqrt(1-t))-np.arcsin(np.sqrt(t))))
    p = p*p
    X = np.array([npr.binomial(1,p) for i in range(n)])
    return p,X

def score_tardos(x,y,p):
    return np.sum((1-2*(x*(1-y)+y*(1-x)))*((np.sqrt((1-p)/p))**(2*y-1)))

In [44]:
p,X = matrice_identifiants(n, m, t)
score_tardos(npr.rand(m), X[0], p)

-3.0651202460295366

## Monte Carlo naif pour trouver la p-value
Soit $y$ un identifiant trouvé dans une copie pirate et $x$ l'identifiant d'un utilisateur.  
On suppose les $p_i$ connus.  
Quelle est la probabilité $p_x$ qu'un innocent ai un score supérieur à celui de x ?  
Pour répondre à cette question nous utiliserons un estimateur Monte Carlo naïf :
$$\hat{p_x}=\frac{1}{N}\overset{N}{\underset{k = 1}\sum}{\mathbb{1}_{S\left(X_k\right)\geq S(x)}}\ \ \text{avec}\ \ X_k \overset{iid}{\sim}\left[\mathcal{Bernoulli}(p_i)\right]_{1\leq i\leq m}$$

In [46]:
def MC_naif(N,p,x,y):
    score = np.array([score_tardos(npr.binomial(1,p),y,p) for i in range(N)])
    return np.average(score >= score_tardos(x,y,p))

In [47]:
MC_naif(10000, p, X[0], npr.rand(m))

0.8491